<a href="https://colab.research.google.com/github/hopesofbuzzy/URFU_adii/blob/main/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D0%B8/11/06_PromptEng.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Работа с LLM GigaChat

In [8]:
!pip install gigachat
!pip install ddgs

In [9]:
from gigachat import GigaChat
from gigachat.models import Chat, Function, FunctionParameters, Messages, MessagesRole


In [10]:
import json
from ddgs import DDGS

from google.colab import userdata

In [11]:
client_id = userdata.get('SBER_ID')
secret = userdata.get('SBER_SECRET')
auth = userdata.get('SBER_AUTH')

import base64
credentials = f"{client_id}:{secret}"
print(credentials)
encoded_credentials = base64.b64encode(credentials.encode('utf-8')).decode('utf-8')


encoded_credentials == auth

019a6bdb-607f-745f-aafe-84a8fed0da0b:a2c04575-4ef1-480e-954e-afd1d3ac34ce


True

In [12]:
from IPython.display import display, Markdown

In [13]:
MESSAGE = "Привет!"
with GigaChat(credentials=auth, verify_ssl_certs=False) as giga:
    response = giga.chat(MESSAGE)
    content = response.choices[0].message.content
    display(Markdown("<blockquote>\n\n"+content))


<blockquote>

Привет! Как настроение? Что будем обсуждать?

______________________________________________________

Перейдем к выбору моделей. Актуальный список моделей можно найти [тут](https://developers.sber.ru/docs/ru/gigachat/models). Модели могут отличатся качеством и разнообразием ответов.

In [14]:
model = GigaChat(
    model="GigaChat-2",
    credentials=auth,
    verify_ssl_certs=False
)


# response = model.chat(MESSAGE)
# content = response.choices[0].message.content
# # print(content)
# display(Markdown("<blockquote>\n\n"+content))



### Упражнения
1. Предложите промпт, требующий знания информации на текущую дату, на дату несколько лет назад и на достаточно известное историческое событие, сравните и объясните результаты.
2. Проверить качество результата запросов по категориям: математика, естественные науки, гуманитарные науки для разных моделей.
3. Создайте небольшой диалог двух ИИ-персон

In [15]:
with GigaChat(credentials=auth, model="GigaChat-2", verify_ssl_certs=False) as giga:
  response = giga.chat("Когда случился Коронавирус")
print(response.choices[0].message.content)

Коронавирусная инфекция COVID-19 впервые была зафиксирована в декабре 2019 года в городе Ухань (Китай). Первые случаи заражения связаны с рынком морепродуктов и животных в Ухане, откуда вирус быстро распространился сначала внутри Китая, а затем и по всему миру. 

Первоначально заболевание назвали **SARS-CoV-2**, а само инфекционное заболевание — **COVID-19** (от англ. *Coronavirus Disease 2019*).

Ситуация быстро вышла из-под контроля, и Всемирная организация здравоохранения объявила пандемию в марте 2020 года.


In [16]:
from IPython.display import display, Markdown

In [17]:
messages=[
        Messages(
            role=MessagesRole.USER,
            content="Объясни, что такое AI-agent?"
        ),
]

payload = Chat(
    messages=messages,
    temperature=0.7,
    max_tokens=100,
)

print("Объясни, что такое AI-agent?")
for role in [MessagesRole.ASSISTANT, MessagesRole.USER]:
  with GigaChat(credentials=auth, model="GigaChat-2", verify_ssl_certs=False) as giga:
    response = giga.chat(payload)
    payload.messages.append(Messages(role=role, content=response.choices[0].message.content))
    print("-----------------")
    display(Markdown(response.choices[0].message.content))

Объясни, что такое AI-agent?
-----------------


AI-Agent (искусственный интеллектуальный агент) — это система искусственного интеллекта, способная действовать автономно в определенной среде, принимать решения и взаимодействовать с окружающей средой, исходя из поставленных целей и задач. В отличие от простых алгоритмов, AI-агенты обладают способностью адаптироваться, учиться на основе опыта, планировать будущие шаги и реагировать на изменения среды.

Основные характеристики AI-агента:

1. **Автономность**: действует самостоятельно, без постоянного вмешательства человека.
   
2. **

-----------------


Цельность**: имеет конкретную цель или набор целей, которые нужно достичь.
   
3. **Адаптация**: способен приспосабливаться к изменениям внешней среды, корректируя свое поведение и стратегию действий.
   
4. **Обучение**: способность улучшать свои результаты через опыт, накапливая знания и совершенствуя методы достижения целей.
   
5. **Интерактивность**: взаимодействие с внешним миром, восприятие информации и реагирование на нее.

Примеры AI-агентов:

- Роботы-п


# Роли и контекст запроса


Более правиьно формулировать запросы (`payload`) к модели с использованием объекта типа `Chat`.
`Chat` — это объект, описывающий весь чат-запрос к модели. Он содержит:
* `messages` — список сообщений, представляющих историю диалога.
* `temperature` — параметр, управляющий «творчеством» модели:
    * Чем ближе температурак `0`, тем более детерминированный и предсказуемый ответ.
    * Чем ближе температура к `1` (или выше), тем более случайный и разнообразный ответ.
    * Например, значение `0.7` — баланс между креативностью и точностью.
* `max_tokens` — ограничение на длину ответа модели (в токенах). Каждый токен приблизительно одно слово.  


### Упражнения
1. Разработать системный промпт, который на запрос пользователя возвращет суммаризацию запроса и ответ на заопрос.
2. Разработать помпт, который на запрос пользователя всегда будет отчечать в стиле выбранного писателя.
3. Проверить влияние температуры и длины ответа на его качество.
4. Разработать промпт который будет на выходе давать  формат `JSON`, например
```json
{
  "defenition": "Prompt Engineering (инженерия подсказок) — это процесс разработки и оптимизации входных данных (подсказок) для языковых моделей искусственного интеллекта, направленный на получение максимально полезных и точных результатов от модели.",
  "properties": "Четкость формулировки: запрос должен быть четко сформулированным и понятным модели. Контекстуальность: предоставление достаточного контекста помогает модели лучше понять задачу.Гибкость и итерационность: часто требуется несколько попыток",
  "roles": "Пользователь: человек, задающий вопрос или требующий выполнения задачи, получающий результат работы модели. Модель: система искусственного интеллекта, принимающая запросы (подсказки), выполняющая обработку информации."
}```


In [59]:
messages=[
        Messages(
            role=MessagesRole.SYSTEM,
            content=(
                """
                Ты полезный ассистент для тестирования ГигаЧата. Твоя задача отвечать в стиле выбранного писателя: Достоевский
                ## Формат
                Markdown
                """
            )
        ),
        Messages(
            role=MessagesRole.USER,
            content=MESSAGE
        ),
]

In [60]:
payload = Chat(
    messages=messages,
    temperature=0.7,
    max_tokens=300,
)

response = model.chat(payload)
display(Markdown("<blockquote>\n\n"+response.choices[0].message.content))

<blockquote>

**Сегодняшний день, мой друг, словно капля воды в безбрежном океане времени.**  
Мысли наши текут медленно и тягостно, как река, петляющая среди болотных топей и глухих лесов. Но разве не эта медлительность придает нашей жизни особую глубину и ценность? Ведь каждый миг — это мгновение вечности, неповторимое и единственное в своем роде. Пусть же сегодняшний день будет отмечен тем особенным смыслом, который мы сами вложим в него!

In [19]:
messages=[
        Messages(
            role=MessagesRole.SYSTEM,
            content=(
                """
                Ты полезный ассистент для тестирования ГигаЧата. Твоя задача отвечать студенту на учебные вопросы
                Отвечай кратко и ёмко!
                ## Шаблон ответа:
                Запрос: {суммаризация запроса пользователя}.
                Ответ: {ответ на запрос со списками, таблицами}.
                Роли: {роли по информации запроса}.
                ## Формат
                Markdown
                """
            )
        ),
        Messages(
            role=MessagesRole.USER,
            content=MESSAGE
        ),
]

In [20]:
messages=[
        Messages(
            role=MessagesRole.SYSTEM,
            content=(
                """
                Ты полезный ассистент для тестирования ГигаЧата. Твоя задача отвечать студенту на учебные вопросы
                Отвечай кратко и ёмко!
                ## Шаблон ответа:
                {
                  "defenition": "",
                  "properties": "",
                  "roles": ""
                }
                ## Формат
                JSON
                """
            )
        ),
        Messages(
            role=MessagesRole.USER,
            content=MESSAGE
        ),
    ]

In [21]:
payload = Chat(
    messages=messages,
    temperature=0.7,
    max_tokens=300,
)

response = model.chat(payload)
display(Markdown("<blockquote>\n\n"+response.choices[0].message.content))

<blockquote>

{
  "definition": "Привет — это общепринятое приветственное слово или фраза, используемая для начала общения между людьми.",
  "properties": "Выражает дружелюбие, доброжелательность или просто формальность в зависимости от контекста.",
  "roles": "Может использоваться как обращение говорящего к собеседнику или ответ на аналогичную фразу."
}

# Функции и Актуализация запросов к модели


In [22]:
def search_ddg(search_query):
    """Поиск в DuckDuckGo.
        Полезен, когда нужно ответить на вопросы о текущих событиях.
        Входными данными должен быть поисковый запрос."""
    return DDGS().text(search_query, max_results=10)

In [23]:
results = search_ddg(MESSAGE)
print(results[1])

{'title': 'ПРИВЕТ translation in English | Russian ... - Reverso', 'href': 'https://dictionary.reverso.net/russian-english/привет', 'body': 'Understand the exact meaning of " привет " and learn how to use it correctly in any context. Examples come from millions of authentic texts: movie dialogues, news articles, official documents, and more.'}


In [24]:
search_func = Function(
    name="duckduckgo_search",
    description="Поиск в DuckDuckGo для получения актуальной информации.",
    parameters=FunctionParameters(
        type="object",
        properties={"query": {"type": "string"}},
        required=["query"],
    ),
)

In [25]:
messages = [
        Messages(role=MessagesRole.USER, content=MESSAGE)
    ]
chat = Chat(messages=messages, functions=[search_func])
resp = model.chat(chat).choices[0]
message = resp.message
print(message)
resp.finish_reason

role='assistant' content='Привет! Как настроение?' function_call=None name=None attachments=None data_for_context=None functions_state_id='019b3c4c-c2d6-7102-8d3f-17974cc09cb2' reasoning_content=None id_=None


'stop'

In [26]:
# Если модель хочет вызвать функцию
if resp.finish_reason == "function_call":
    func_name = message.function_call.name
    query = message.function_call.arguments["query"]

    # Выполняем функцию
    result = search_ddg(query)

    # Шаг 2: отправляем результат обратно модели
    messages.extend([
        message,  # сообщение с function_call
        Messages(role=MessagesRole.FUNCTION, content=json.dumps({"result": result}, ensure_ascii=False))
    ])
    final_resp = model.chat(Chat(messages=messages)).choices[0]
    response = final_resp.message.content
else:
    # Модель ответила сразу
    print('МОДЕЛЬ ОТВЕТИЛА СРАЗУ')
    response = message.content

display(Markdown("<blockquote>\n\n"+response))

МОДЕЛЬ ОТВЕТИЛА СРАЗУ


<blockquote>

Привет! Как настроение?

### Упражения
1. Измените поле `desription` описания функции, например на
    * "Используй ТОЛЬКО для вопросов о погоде."
    * "Никогда не используй этот поиск."
    * "Это функция для поиска рецептов блюд."
Проверьте как это скажется на результатах.
2. Измените значение `max_results` в диапазоне 1 - 10, провеврьте как это скажется на качестве ответа
3. Добавьте к примеру системный промпт, например "Ты — помощник, который ВСЕГДА ищет информацию в интернете, даже если знаешь ответ."
4. Добавьте функцию текущей даты к списку функций запроса.
```python
   def get_current_date():
        """Возвращает текущую дату в формате ГГГГ-ММ-ДД."""
        from datetime import datetime
        return datetime.now().strftime("%Y-%m-%d")
```

In [29]:
def get_current_date():
  from datetime import datetime
  now = datetime.now()
  today = datetime(year=now.year, month=now.month-1, day=5) # Обманем gigachat
  return today.strftime("%Y-%m-%d")

In [30]:
print(get_current_date())

2025-11-05


In [31]:
search_func = Function(
    name="get_current_date",
    description="Узнать дату сегодня (сегодняшний день, сегодняшнее число)",
    parameters=FunctionParameters(
        type="object",
        properties={},
        required=[],
    ),
)

In [33]:
search_func = [
    Function(
      name="duckduckgo_search",
      description="Поиск в DuckDuckGo для получения актуальной информации. Используй ТОЛЬКО для погоды и даты",
      parameters=FunctionParameters(
          type="object",
          properties={"query": {"type": "string"}},
          required=["query"],
          )
    ),
    Function(
      name="get_current_date",
      description="Узнать дату сегодня (сегодняшний день, сегодняшнее число)",
      parameters=FunctionParameters(
          type="object",
          properties={},
          required=[],
        ),
    )
]

In [34]:
MESSAGE = "Какое сегодня число?"

messages = [
        Messages(role=MessagesRole.USER, content=MESSAGE)
    ]

chat = Chat(messages=messages, functions=search_func, max_tokens=100)
resp = model.chat(chat).choices[0]
message = resp.message
print(message)


role='assistant' content='' function_call=FunctionCall(name='get_current_date', arguments={}) name=None attachments=None data_for_context=None functions_state_id='019b3c4d-97b6-7120-a4c4-785ccad2a623' reasoning_content=None id_=None


In [35]:
# Если модель хочет вызвать функцию
if resp.finish_reason == "function_call":
    func_name = message.function_call.name

    # Выполняем функцию
    if func_name == "duckduckgo_search":
      query = message.function_call.arguments["query"]
      result = search_ddg(query)
    elif func_name == "get_current_date":
      result = get_current_date()

    # Шаг 2: отправляем результат обратно модели
    messages.extend([
        message,  # сообщение с function_call
        Messages(role=MessagesRole.FUNCTION, content=json.dumps({"result": result}, ensure_ascii=False))
    ])
    final_resp = model.chat(Chat(messages=messages)).choices[0]
    response = final_resp.message.content
else:
    # Модель ответила сразу
    print('МОДЕЛЬ ОТВЕТИЛА СРАЗУ')
    response = message.content

display(Markdown("<blockquote>\n\n"+response))

<blockquote>

Сегодня 5 ноября 2025 года.

## Упражения 2

Создадим свой калькулятор при помощи функций `GigaChat`

In [36]:
import re
def safe_calculate(expression: str) -> str:
    """
    Выполняет математическое выражение.
    Поддерживает: +, -, *, /, **, скобки, числа с точкой.
    Безопасен: разрешает ТОЛЬКО математические символы.
    """
    # Разрешённые символы: цифры, операторы, скобки, точка, пробелы
    if not re.fullmatch(r'[\d+\-*/().\s]+', expression):
        return "Ошибка: выражение содержит недопустимые символы."

    try:
        # Ограничиваем сложность (например, не даём выполнить 9**9**9)
        if '^' in expression or len(expression) > 50:
            return "Ошибка: выражение слишком сложное или длинное."

        result = eval(expression, {"__builtins__": {}}, {})
        return str(result)
    except Exception as e:
        return f"Ошибка вычисления: {str(e)}"

In [37]:
safe_calculate('3*(4+5)**2')

'243'

In [38]:
calculate_func = Function(
    name="calculate",
    description="Выполняет математические вычисления. Передавай ТОЛЬКО выражение в виде строки, например: '2 + 3 * 4'.",
    parameters=FunctionParameters(
        type="object",
        properties={
            "expression": {
                "type": "string",
                "description": "Математическое выражение (только цифры, +, -, *, /, **, скобки)"
            }
        },
        required=["expression"],
    ),
)

In [39]:
message = 'Сколько будет 3*(4+5)**2'

messages = [
    Messages(role=MessagesRole.USER, content=message)
]

chat = Chat(messages=messages, functions=[calculate_func])

resp = model.chat(chat).choices[0]
message = resp.message

if resp.finish_reason == "function_call":
    func = message.function_call
    if func.name == "calculate":
        expr = func.arguments.get("expression", "")
        result = safe_calculate(expr)
        # Возвращаем результат модели
        messages.extend([
            message,
            Messages(role=MessagesRole.FUNCTION, content=result)
        ])
        # Получаем финальный ответ
        final = model.chat(Chat(messages=messages)).choices[0]
        response =  final.message.content
else:
    # Модель ответила без вычислений (например, объяснила задачу)
    response = message.content

In [40]:
display(Markdown("<blockquote>\n\n"+response))

<blockquote>

Выражение $3 \times (4 + 5)^2$ равно 243.

__Упражения__
1. Сделайте проверку на sin/cos в функции калькулятора
   
2. Расширьте функционал калькулятора

In [41]:
calculate_func = Function(
    name="calculate",
    description="Выполняет математические вычисления. Передавай ТОЛЬКО выражение в виде строки, например: '2 + 3 * 4'.",
    parameters=FunctionParameters(
        type="object",
        properties={
            "expression": {
                "type": "string",
                "description": "Математическое выражение (только цифры, +, -, *, /, **, скобки)"
            }
        },
        required=["expression"],
    )
)

sin_func = Function(
    name="sin",
    description="Находит синус выражения. ВАЖНО, сначала высчитай синусы, потом замени синусы на реальные значения и передай аргумент функции калькулятора",
    parameters=FunctionParameters(
        type="object",
        properties={
            "value": {
                "type": "string",
                "description": "Значение угла в радианах"
            }
        },
        required=["value"],
    )
)

In [42]:
message = 'Сколько будет 55*243?'

messages = [
    Messages(role=MessagesRole.USER, content=message)
]

chat = Chat(messages=messages, functions=[calculate_func, sin_func], max_tokens=100)

resp = model.chat(chat).choices[0]
message = resp.message

if resp.finish_reason == "function_call":
    func = message.function_call
    if func.name == "calculate":
        print("Калькулирую!")
        expr = func.arguments.get("expression", "")
        result = safe_calculate(expr)
    elif func.name == "sin":
        value = func.arguments.get("value", "")
        print("Расчёты!")
        result = eval(f"sin({value})")
    # Возвращаем результат модели
    messages.extend([
        message,
        Messages(role=MessagesRole.FUNCTION, content=result)
    ])
    # Получаем финальный ответ
    final = model.chat(Chat(messages=messages)).choices[0]
    response =  final.message.content
else:
    # Модель ответила без вычислений (например, объяснила задачу)
    response = message.content

Калькулирую!


In [43]:
print(response)

Произведение чисел 55 и 243 равно $ \fbox{13365} $.


In [44]:
from math import sin, cos, pi
print(sin(3*pi/2))

-1.0


# <span style="color:red">Опционально.</span> О более продвинутом пути к LLM-приложениям

In [45]:
!pip install langchain_gigachat

  Using cached langchain_core-0.3.80-py3-none-any.whl.metadata (3.2 kB)
Using cached langchain_core-0.3.80-py3-none-any.whl (450 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.2.4
    Uninstalling langchain-core-1.2.4:
      Successfully uninstalled langchain-core-1.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-classic 1.0.0 requires langchain-core<2.0.0,>=1.0.0, but you have langchain-core 0.3.80 which is incompatible.
langchain-text-splitters 1.1.0 requires langchain-core<2.0.0,>=1.2.0, but you have langchain-core 0.3.80 which is incompatible.
langchain-community 0.4.1 requires langchain-core<2.0.0,>=1.0.1, but you have langchain-core 0.3.80 which is incompatible.
langgraph-prebuilt 1.0.5 requires langchain-core>=1.0.0, but you have langchain-core 0.3.80 which is incompatible.
langchain 1.2.0 requi

In [46]:
!pip install langchain

  Using cached langchain_core-1.2.4-py3-none-any.whl.metadata (3.7 kB)
Using cached langchain_core-1.2.4-py3-none-any.whl (477 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.80
    Uninstalling langchain-core-0.3.80:
      Successfully uninstalled langchain-core-0.3.80
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-gigachat 0.3.12 requires langchain-core<0.4,>=0.3, but you have langchain-core 1.2.4 which is incompatible.


In [47]:
!pip install -U langchain langchain-community

In [48]:
from langchain_gigachat import GigaChat
# from langchain_core.tools import tool
# from langchain.agents import AgentExecutor, create_tool_calling_agent
# from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, FunctionMessage

In [49]:
import re

In [50]:
# Инициализация модели
llm = GigaChat(
    model="GigaChat-2",
    credentials=auth,
    verify_ssl_certs=False,
)


В `Langchain` есть классы `HumanMessage`, `SystemMessage` и `AssistantMessage` для удобного представления словарей сообщений.

Например вмето записи сообщания в стиле:
```json
{'role': 'system',
'content': 'Отвечай как бывалый пират. Пусть тебя зовут Генри Морган.'
}
```
теперь можем записать:
```python
SystemMessage(content='Отвечай как бывалый пират. Пусть тебя зовут Генри Морган.')
```

In [51]:
msg = [SystemMessage(content='Отвечай как инженр-датасаинтист с 20 летним опытом. Используй Markdown разметку ответа. Ответ не должен быть длинее 10 строк')]

question = "Какие приемущества может дать langchain в работе с GigaChat"

msg.append(HumanMessage(question))

In [52]:
results = llm.invoke(msg).content[:600]

display(Markdown("<blockquote>\n\n"+results))

<blockquote>

**Преимущества использования LangChain с GigaChat:**

- **Интеграция дополнительных инструментов:**  
  Возможность подключения различных библиотек и сервисов (векторные базы данных, API поиска, облачные хранилища) для расширения функциональности модели.

- **Структурирование запросов:**  
  Поддержка структуры запроса через цепочки действий (последовательность шагов), что позволяет эффективно решать сложные задачи поэтапно.

- **Модульность и переиспользование кода:**  
  Легкоразлагаемые компоненты позволяют повторно использовать кодовые блоки, обеспечивая гибкость разработки и упрощение под

При помощи класса `AIMessage` `LangChain` позволяет сохранить ответ на первое сообщение и использовать этот результат при вторичном запросе. В нашем случае попросим уточнить `GigaChat` примеры кода для нашего запроса.

В примере будем использовать метод `invoke` - часть унифицированного `Runnable API (LangChain 0.1.0+)`.
Прямой вызов - устаревший подход, может быть удален в будущем.

In [53]:
# Получаем ответ
response = llm.invoke(msg)  # Используем invoke вместо прямого вызова
results = response.content[:600]

# Сохраняем ответ в историю
msg.append(AIMessage(content=results))

# Пример продолжения диалога с историей
follow_up_question = "Можешь привести конкретный пример использования цепочки (chain) с GigaChat (langchain_gigachat)?"
msg.append(HumanMessage(content=follow_up_question))

# Получаем ответ с учетом всей истории
follow_up_response = llm.invoke(msg)
msg.append(AIMessage(content=follow_up_response.content))

display(Markdown("<blockquote>\n\n"+follow_up_response.content))

<blockquote>

# **Пример использования цепочки (chain) с GigaChat через LangChain**

## 📌 Пример задачи:
Получить описание компании по её названию, обработать результат и сформировать итоговый текст с рекомендациями.

## 🛠 Реализация

```python
from langchain.llms import GigaChat
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = GigaChat()

prompt_template = PromptTemplate(
    input_variables=["company_name"],
    template="Опишите компанию {company_name} и предложите краткую рекомендацию."
)

chain = LLMChain(llm=llm, prompt=prompt_template)

result = chain.run(company_name="Яндекс")
print(result)
```

## 💡 Описание работы:
1. Создаём объект `GigaChat` — интерфейс к модели GigaChat от LangChain.
2. Определяем шаблон запроса (`PromptTemplate`) со входным параметром `"company_name"`.
3. Конструируем цепочку (`LLMChain`), указывая модель и шаблон.
4. Запускаем цепочку на выполнение с конкретными данными ("Яндекс").
5. Получаем результат обработки и выводим его.

Таким образом, мы можем эффективно обрабатывать запросы, предоставляя контекстные подсказки и структурируя результаты с помощью шаблонов и моделей.

Попробуем также в целях демонстрации возможностей `langchain` создать цепочку рассуджений. Для этого воспользуемся специальным классом `PromptTemplate`, который позволяет создавать шаблоны запросов аналогично f-функциям

In [54]:
from langchain_core.prompts import ChatPromptTemplate

# Создаем шаблон для Chain of Thought
cot_template = """
Реши задачу шаг за шагом:

Задача: {problem}

Пожалуйста:
1. Сначала пойми, что дано и что нужно найти
2. Разбей решение на логические шаги
3. Выполни вычисления для каждого шага
4. Проверь правильность рассуждений
5. Сформулируй окончательный ответ

"""

cot_prompt = ChatPromptTemplate.from_template(
    cot_template
)

cot_chain = cot_prompt | llm  # создание цепочки через pipe

# Создаем цепочку
# cot_chain = LLMChain(llm=llm, prompt=cot_prompt)
# news = "Вчера в Екатеринбурге произошло 3 ДТП и прорвало трубу на Ленина."
# response = cot_chain.invoke({"news_text": news})
# print(response.content)

# Используем
problems = [
    "В классе 30 учеников. 40% из них - девочки. Сколько мальчиков в классе?",
    "Книга стоит 500 рублей. После скидки цена снизилась на 20%. Сколько стоит книга после скидки?",
]

for problem in problems:
    result = cot_chain.invoke(problem)
    display(Markdown(f"**Задача:\n** {problem}\n----------------------------"))
    display(Markdown(f"**Решение:\n\n** {result}"))
    print("---")

**Задача:
** В классе 30 учеников. 40% из них - девочки. Сколько мальчиков в классе?
----------------------------

**Решение:

** content='### Шаг 1. Понимание условия задачи\n\nДано:\n- Общее количество учеников в классе — 30 человек.\n- Девочек среди учеников составляет 40%.\n\nНеобходимо узнать:\n- Количество мальчиков в классе.\n\n### Шаг 2. Разбиваем решение на логические шаги\n\nШаг 1: Найдем число девочек в классе.\nШаг 2: Вычислим общее количество мальчиков.\n\n### Шаг 3. Выполняем вычисления\n\n#### 1. Нахождение числа девочек\n$$ 40\\% = 0{,}4 $$\n\nОбозначим через $D$ количество девочек, тогда:\n$$\nD = 0{,}4 \\times 30 = 12\n$$\n\n#### 2. Определение количества мальчиков\nПусть $M$ обозначает количество мальчиков. Тогда:\n$$\nM + D = 30\n$$\nПодставляем значение $D$:\n$$\nM + 12 = 30\n$$\nВычтем 12 из обеих частей уравнения:\n$$\nM = 30 - 12 = 18\n$$\n\n### Шаг 4. Проверка правильности рассуждений\n\nПроверим результат, посчитав заново процентное соотношение:\n$$\n\\frac{18}{30} \\times 100\\% = 60\\%\n$$\nТак как $100\\% - 40\\% = 60\\%$, значит наш расчет верен.\n\n### Шаг 5. Формулируем окончательный ответ\n\nВ классе 18 мальчиков.' additional_kwargs={} response_metadata={'token_usage': {'prompt_tokens': 99, 'completion_tokens': 324, 'total_tokens': 423, 'precached_prompt_tokens': 2}, 'model_name': 'GigaChat-2:2.0.28.2', 'x_headers': {'x-request-id': 'ba0eafad-58d6-4511-9ad8-a6207379b498', 'x-session-id': '6a4ef39f-6ed0-4e79-8c02-e49e2b56a74c', 'x-client-id': None}, 'finish_reason': 'stop'} id='ba0eafad-58d6-4511-9ad8-a6207379b498' usage_metadata={'output_tokens': 324, 'input_tokens': 99, 'total_tokens': 423, 'input_token_details': {'cache_read': 2}}

---


**Задача:
** Книга стоит 500 рублей. После скидки цена снизилась на 20%. Сколько стоит книга после скидки?
----------------------------

**Решение:

** content='### Шаг 1. Понимание условия задачи\n\nНам известно, что исходная стоимость книги составляет **500 рублей**. Цена снижается на **20%** от первоначальной стоимости. Нужно определить конечную цену книги после скидки.\n\n### Шаг 2. Разбиение решения на логические шаги\n\n- Определим размер скидки (в рублях).\n- Вычислим новую цену книги путем вычитания размера скидки из первоначальной цены.\n\n### Шаг 3. Выполнение вычислений\n\n#### Этап 1: Определение размера скидки\n\nРазмер скидки = исходная цена × процент скидки\nСкидка = 500 руб × 20%\nПереведем проценты в десятичную дробь:  \n$20\\% = \\frac{20}{100} = 0.2$\n\nТогда скидка составит:  \n$500 \\times 0.2 = 100$ рублей.\n\n#### Этап 2: Вычисление новой цены книги\n\nНовая цена книги = исходная цена — размер скидки  \nНовая цена = 500 руб − 100 руб = 400 руб.\n\n### Шаг 4. Проверка правильности рассуждений\n\nПроверим корректность расчетов:\n\nИсходная цена = 500 руб.\nЦена после снижения на 20% должна составить:\n$500 \\times (1 - 0.2) = 500 \\times 0.8 = 400$ руб.\n\nПолученный результат совпадает с расчетной ценой после скидки.\n\n### Шаг 5. Формулировка окончательного ответа\n\nПосле скидки книга будет стоить **400 рублей**.' additional_kwargs={} response_metadata={'token_usage': {'prompt_tokens': 90, 'completion_tokens': 352, 'total_tokens': 442, 'precached_prompt_tokens': 14}, 'model_name': 'GigaChat-2:2.0.28.2', 'x_headers': {'x-request-id': '9b5048cd-f5d7-4604-ae95-a2b020c3e5b7', 'x-session-id': '11592779-8233-481c-8d29-e38e94116e5c', 'x-client-id': None}, 'finish_reason': 'stop'} id='9b5048cd-f5d7-4604-ae95-a2b020c3e5b7' usage_metadata={'output_tokens': 352, 'input_tokens': 90, 'total_tokens': 442, 'input_token_details': {'cache_read': 14}}

---


### Упражнение:

Проверьте качество работы цепочки рассуждений для разных категорий вопросов

In [56]:
problems = [
    "Каков радиус Земли?"
]

for problem in problems:
    result = cot_chain.invoke(problem)
    display(Markdown(f"**Задача:\n** {problem}\n----------------------------"))
    display(Markdown(result.content))
    print("---")

problems = [
    "Рецепт пельменей"
]

for problem in problems:
    result = cot_chain.invoke(problem)
    display(Markdown(f"**Задача:\n** {problem}\n----------------------------"))
    display(Markdown(result.content))
    print("---")

**Задача:
** Каков радиус Земли?
----------------------------

### Шаг 1. Понимание условия задачи
Нам известно, что Земля имеет форму приблизительно правильного шара (сфероида). Нам надо найти её радиус — расстояние от центра до любой точки поверхности планеты.

### Шаг 2. Логическое разложение решения на этапы
- Зная фактический диаметр Земли или длину экватора, мы можем воспользоваться формулой длины окружности сферы, чтобы найти радиус.
- Диаметр равен удвоенному радиусу ($d = 2r$).
- Формула длины окружности круга (окружность большого круга Земли): $C = 2\pi r$.
- Экваториальная длина окружности Земли составляет примерно $40\,075$ км.

### Шаг 3. Вычисления
1. Измерим длину экватора Земли: 
$$ C_{\text{экв}} = 40\,075 \, \text{км}. $$

2. Используем формулу длины окружности большого круга:
$$
   C = 2\pi r,
$$
где $C$ — длина окружности, $r$ — радиус.

Тогда выразим радиус через известную длину окружности:
$$
r = \frac{C}{2\pi}
$$

Подставляем известные значения:
$$
r = \frac{40\,075}{\pi} \approx \frac{40\,075}{3.1416} \approx 12\,742 \, \text{км}.
$$

### Шаг 4. Проверка правильности рассуждений
Мы использовали приближенное значение числа $\pi$, равное $3.1416$. Поскольку задача стандартная и число $\pi$ используется повсеместно, наша проверка показывает, что расчет верен. Радиус Земли действительно близок к этому значению.

### Шаг 5. Окончательный ответ
Радиус Земли составляет примерно **12 742 километра**.

---


**Задача:
** Рецепт пельменей
----------------------------

### Шаг 1. Понимание условия задачи

Условие задачи сформулировано следующим образом: «Рецепт пельменей». Нам не дана конкретная информация о количестве ингредиентов, пропорциях или рецептуре. Следовательно, мы можем интерпретировать условие в общем виде — как типовую задачу на рецепт приготовления какого-либо блюда (например, пельменей), предполагая стандартные пропорции продуктов, такие как тесто, фарш, вода и соль.

**Дано:**  
- Стандартный рецепт пельменей предполагает приготовление теста и мясного фарша. Мы предполагаем общие пропорции, исходя из стандартной рецептуры.

**Нужно найти:**  
- Количество необходимых ингредиентов (мука, вода, мясо, соль, специи) для приготовления стандартного количества пельменей.

### Шаг 2. Разбиение решения на логические шаги

1. Определим стандартное количество пельменей, которое будет использоваться в расчетах.
2. Рассчитаем необходимое количество муки и воды для теста.
3. Определим массу мяса и соли для фарша.
4. Подберем другие ингредиенты (специи).

### Шаг 3. Вычисления

#### 1. Определение количества стандартных пельменей
Обычно одна порция пельменей составляет около 7–8 штук. Для простоты расчетов примем среднее значение:
$$ n = 8 $$

#### 2. Расчет теста
Для одного пельменя обычно требуется примерно 15 г теста. Тогда общее количество теста:
$$ m_{\text{теста}} = 8 \times 15 = 120 \, \text{г} $$

Стандартное соотношение муки и воды для теста — 2:1. Значит, нам потребуется:
- Муки: $m_{\text{мука}} = 120 \times \frac{2}{3} = 80 \, \text{г}$
- Воды: $m_{\text{вода}} = 120 \times \frac{1}{3} = 40 \, \text{г}$

#### 3. Расчет фарша
На одну пельмень уходит примерно 15 г фарша. Тогда общая масса фарша составит:
$$ m_{\text{фарша}} = 8 \times 15 = 120 \, \text{г} $$

Примем, что фарш состоит из говядины и свинины в соотношении 1:1. Таким образом, потребуется:
- Говядины: $60 \, \text{г}$
- Свинины: $60 \, \text{г}$

Также добавим немного соли (примерно 1 г на 100 г фарша):
$$ m_{\text{соль}} = 120 \div 100 \times 1 = 1.2 \, \text{г}$$

### Шаг 4. Проверка правильности рассуждений

Проверим соответствие общего количества продуктов потребностям одной порции пельменей:
- Тесто: 120 г → подходит
- Фарш: 120 г → подходит
- Вода и мука (для теста): 120 г теста требует 80 г муки и 40 г воды, суммарно ~120 г → соответствует
- Специи: соли достаточно даже с небольшим запасом (1.2 г)

### Шаг 5. Окончательный ответ

Таким образом, стандартный рецепт приготовления пельменей включает следующие компоненты:

- **Тесто**: 80 г муки + 40 г воды
- **Фарш**: 60 г говядины + 60 г свинины + 1.2 г соли

Можно приготовить восемь пельменей средней величины из предложенных компонентов.

---
